In [18]:
# doping profile

import numpy as np
import matplotlib.pyplot as plt

def band_initialization(NA, ND, ni, q, kB, T):
    dN = ND - NA
    V = np.zeros_like(dN)
    for i in range(len(V)):
        if abs(dN[i]) >= ni:
            V[i] = np.sign(dN[i]) * kB * T / q * np.log(abs(dN[i]) / ni)
    return V

def heaviside(x):
    return np.heaviside(x, 1.0)  # same as MATLAB's heaviside with 1 at x=0

def doping_profiles(profile_type, x):
    NA = np.zeros_like(x)
    ND = np.zeros_like(x)

    #NA=1e15, ND=1e16, can not calculate when exchange NA and ND.
    if profile_type == 'abrupt':
        NA[:] = NA1 * 1e6  # convert to 1/m^3
        ND = np.where(x >= 0, ND1, 0) * 1e6

    elif profile_type == 'linear':
        alpha = 1e19 * 100e6  # 1/m^4
        for i in range(len(x)):
            if x[i] <= 0:
                NA[i] = -alpha * x[i]
            else:
                ND[i] = alpha * x[i]

    elif profile_type == 'hyper_abrupt':
        NA = 1e15 * np.exp(0.5 * (x / 1e-6)) * heaviside(-x)
        ND = 1e15 * np.exp(0.5 * (-x / 1e-6)) * heaviside(x)
        NA *= 1e6
        ND *= 1e6

    elif profile_type == 'abrupt_pnp':
        NA = (heaviside(-x - 1e-6) * NE + heaviside(x + 1e-6) * NC) * 1e6
        ND = (heaviside(x + 1e-6) * NB - heaviside(x - 1e-6) * NB) * 1e6

    elif profile_type == 'abrupt_npn':
        ND = (heaviside(-x - 1e-6) * NE + heaviside(x + 1e-6) * NC) * 1e6
        NA = (heaviside(x + 1e-6) * NB - heaviside(x - 1e-6) * NB) * 1e6

    elif profile_type == 'abrupt_pnpn':
        NA = (heaviside(-x - 2e-6) * 1e18 +
              (heaviside(x - 2e-6) - heaviside(x - 3e-6)) * 0.8e18) * 1e6
        ND = ((heaviside(x + 2e-6) - heaviside(x - 2e-6)) * 5e17 +
              heaviside(x - 3e-6) * 1.2e18) * 1e6

    elif profile_type == 'abrupt_npnp':
        ND = (heaviside(-x - 2e-6) * 1e18 +
              (heaviside(x - 2e-6) - heaviside(x - 3e-6)) * 0.8e18) * 1e6
        NA = ((heaviside(x + 2e-6) - heaviside(x - 2e-6)) * 5e17 +
              heaviside(x - 3e-6) * 1.2e18) * 1e6

    elif profile_type == 'arbitrary':
        NA, ND = my_profile(x)

    else:
        raise ValueError(f"[ERROR]: {profile_type} is not a valid doping profile!")

    return NA, ND

def my_profile(x):
    NA = np.ones_like(x) * 1e15 * 1e6
    ND = np.exp(-(x / 1e-6) ** 2) * 1e17 * 1e6
    return NA, ND

# Declaration of constants
T = 300 # degreeK
eps_0 = 8.854187813e-12 #As/Vm, permittivity 8.854e-14 F/cm(1/(mu_0*c^2))
eps_r = 11.7
epsilon = eps_0 * eps_r
kB = 1.380658e-23 #J/K
q = 1.60217733e-19 #As
ni = 1.45e16 # 1/m^3
Eg = 1.12 # eV



In [19]:
# Domain
xmin, xmax = -5e-6, 5e-6
N = 11
x = np.linspace(xmin, xmax, N)
dx = x[1] - x[0]
xg = np.concatenate(([x.min() - dx], x, [x.max() + dx]))

print(x)
print(dx)
print(xg)

[-5.00000000e-06 -4.00000000e-06 -3.00000000e-06 -2.00000000e-06
 -1.00000000e-06  8.47032947e-22  1.00000000e-06  2.00000000e-06
  3.00000000e-06  4.00000000e-06  5.00000000e-06]
9.999999999999997e-07
[-6.00000000e-06 -5.00000000e-06 -4.00000000e-06 -3.00000000e-06
 -2.00000000e-06 -1.00000000e-06  8.47032947e-22  1.00000000e-06
  2.00000000e-06  3.00000000e-06  4.00000000e-06  5.00000000e-06
  6.00000000e-06]


In [20]:
# Newton Raphson paratmeters
maxIter = 500
alpha = 1e-8

In [21]:
#abrupt junction, ND1 should be bigger than NA1
NA1=1e15
ND1=5e15
get_doping_profile = 'abrupt'
NA, ND = doping_profiles(get_doping_profile, x)
print(NA*1e-6,ND*1e-6)

[1.e+15 1.e+15 1.e+15 1.e+15 1.e+15 1.e+15 1.e+15 1.e+15 1.e+15 1.e+15
 1.e+15] [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 5.e+15 5.e+15 5.e+15 5.e+15 5.e+15
 5.e+15]


In [22]:
def doping_profiles(x, NA1, ND1):
    NA = np.ones_like(x) * NA1 * 1e6  # convert to 1/m^3
    ND = np.where(x >= 0, ND1, 0) * 1e6
    return NA, ND

print(NA,ND)

[1.e+21 1.e+21 1.e+21 1.e+21 1.e+21 1.e+21 1.e+21 1.e+21 1.e+21 1.e+21
 1.e+21] [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 5.e+21 5.e+21 5.e+21 5.e+21 5.e+21
 5.e+21]


In [23]:
# Initial charge carriers
n = ND.copy()
p = NA.copy()

print(n)
print(p)

[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 5.e+21 5.e+21 5.e+21 5.e+21 5.e+21
 5.e+21]
[1.e+21 1.e+21 1.e+21 1.e+21 1.e+21 1.e+21 1.e+21 1.e+21 1.e+21 1.e+21
 1.e+21]


In [24]:
# Initial potential
V = np.zeros_like(xg)
print(V)
Vinit = band_initialization(NA, ND, ni, q, kB, T)
print(Vinit)
'''
def band_initialization(NA, ND, ni, q, kB, T):
    dN = ND - NA
    V = np.zeros_like(dN)
    for i in range(len(V)):
        if abs(dN[i]) >= ni:
            V[i] = np.sign(dN[i]) * kB * T / q * np.log(abs(dN[i]) / ni)
    return V
'''
V[1:N+1] = Vinit
V[0] = V[1]
V[-1] = V[-2]
V_old = V.copy()
print(V)
print(V_old)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[-0.28802824 -0.28802824 -0.28802824 -0.28802824 -0.28802824  0.32386694
  0.32386694  0.32386694  0.32386694  0.32386694  0.32386694]
[-0.28802824 -0.28802824 -0.28802824 -0.28802824 -0.28802824 -0.28802824
  0.32386694  0.32386694  0.32386694  0.32386694  0.32386694  0.32386694
  0.32386694]
[-0.28802824 -0.28802824 -0.28802824 -0.28802824 -0.28802824 -0.28802824
  0.32386694  0.32386694  0.32386694  0.32386694  0.32386694  0.32386694
  0.32386694]


In [25]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags

for k in range(1, maxIter + 1):
    n = ni * np.exp(q * V[1:N+1] / (kB * T))
    p = ni * np.exp(-q * V[1:N+1] / (kB * T))
    print(f"n[{k}] = {n[k]}")
    rho = q * (p - n + ND - NA)
    print(f"k={k}, rho[{k}] = {rho[k]}")
    drho_dV = -2 * q**2 * ni / (kB * T) * np.cosh(q * V[1:N+1] / (kB * T))
    print(f"drho_dV[{k}] = {drho_dV[k]}")
    d2V_dx2 = (V[0:N] - 2 * V[1:N+1] + V[2:N+2]) / dx**2
    print(f"d2V_dx2[{k}] = {d2V_dx2[k]}")   
    
    Mjj_k = 2 / dx**2 - (1 / epsilon) * drho_dV
    print(f"Mjj_k[{k}] = {Mjj_k[k]}")
    M_off = np.ones(N) / dx**2
    print(f"M_off[{k}] = {M_off[k]}")
    M_k = spdiags([-M_off, Mjj_k, -M_off], [-1, 0, 1], N, N).tocsc()
    print(f"M_k[{k}] = {M_k[k]}")
    R_k = (1 / epsilon) * rho + d2V_dx2
    print(f"R_k[{k}] = {R_k[k]}")
    
    delta_V = np.linalg.solve(M_k.toarray(), R_k)
    print(f"delta_V[{k}] = {delta_V[k]}")
    V[1:N+1] += delta_V
    print(f"V[{N}] = {V[N]}")
    print(f"V[{k}] = {V[k]}")
    if np.linalg.norm(V - V_old, ord=np.inf) < alpha:
        break

    V_old = V.copy()

# Electric field
E = -np.diff(V[:N+1]) / dx
print(E)
Vbi = np.abs(V[-1] - V[0])
print(Vbi)

# Output results
#print(f"Number of Newton-Raphson Iterations: {k}")
#print(f"Built-in Voltage Vbi: {Vbi:.4f} V")

n[1] = 210250000000.00018
k=1, rho[1] = -3.368592115954758e-08
drho_dV[1] = -6197.460914302291
d2V_dx2[1] = 0.0
Mjj_k[1] = 61824517531151.99
M_off[1] = 1000000000000.0004
M_k[1] = <Compressed Sparse Column sparse matrix of dtype 'float64'
	with 3 stored elements and shape (1, 11)>
  Coords	Values
  (0, 0)	-1000000000000.0004
  (0, 1)	61824517531151.99
  (0, 2)	-1000000000000.0004
R_k[1] = -325.17251965424197
delta_V[1] = 4.1750119697801883e-08
V[11] = 0.32386693768730734
V[1] = -0.28802823743861933
n[2] = 210270990429.64108
k=2, rho[2] = -0.015993866432874176
drho_dV[2] = -6196.842248993524
d2V_dx2[2] = 154397602.01141357
Mjj_k[2] = 61818545512268.445
M_off[2] = 1000000000000.0004
M_k[2] = <Compressed Sparse Column sparse matrix of dtype 'float64'
	with 3 stored elements and shape (1, 11)>
  Coords	Values
  (0, 1)	-1000000000000.0004
  (0, 2)	61818545512268.445
  (0, 3)	-1000000000000.0004
R_k[2] = 7706.530789345503
delta_V[2] = 6.184836181655021e-07
V[11] = 0.3238669376873072
V[2] = -